In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals

import ipywidgets as w
from netCDF4 import Dataset 
import time
import pandas as pd
import numpy as np
import calendar

from datetime import datetime
import plotly
import chart_studio.plotly as py 
import plotly.graph_objects as go

import OrbitFileConversions

style1 = {'description_width':'120px'}
layout1 = {'width':'780px'}
OrbitFilename_Text      = w.Text(value="", description='Orbit file name:', style=style1, layout=layout1)
TIEGCMfilesFolder_Text  = w.Text(value=DaedalusGlobals.TIEGCM_Files_Path + "TIEGCM_DERIVED_PRODUCTS/", description='TIEGCM files folder:', style=style1, layout=layout1)
ResultFilename_Text     = w.Text(value="", description='Result file name:', style=style1, layout=layout1)
NetCDFcorrection_Text   = w.Text(value="", description='NetCDF files path:', style=style1, layout=layout1)
Regions_Dropdown        = w.Dropdown( options=["AEM", "AFM", "AEE", "AED", "EEJ", "EPB", "SQ", "CF", "PCF"], description='Region of interest: ', style=style1, layout=layout1)
#MagCoordSystem_Dropdown = w.Dropdown( options=["Modified Apex (90km)", "Quasi Dipole"], description='Mag. coord. system: ', style=style1, layout=layout1)

def NetCDFcorrection_Btn_EventListener( b ):
    print( "Not implemented" )

def Convert_AddFieldsToCSVorbit_Btn_Clicked( b ):
    print( "Reading", OrbitFilename_Text.value, "\nWriting", ResultFilename_Text.value  )
    OrbitFileConversions.AddMagneticCoordinates( OrbitFilename_Text.value, ResultFilename_Text.value )
    #if MagCoordSystem_Dropdown.value.startswith( "Quasi" ):
    #    OrbitFileConversions.AddMagneticCoordinates_QuasiDipole( OrbitFilename_Text.value, ResultFilename_Text.value )
    #else:
    #    OrbitFileConversions.AddMagneticCoordinates_ModifiedApex90( OrbitFilename_Text.value, ResultFilename_Text.value )
    print( "Done" )

def Convert_CherryPick_Btn_Clicked( b ):
    # initialize
    print( "Reading from", TIEGCMfilesFolder_Text.value, "and Writing", ResultFilename_Text.value  )
    startSecs = time.time()
    ResultFileIdx = 0
    
    # define limits of the selected region
    minMLT=22; maxMLT=2;  minMagLat=60; maxMagLat=75

    # Create the result file
    resultCDF = Dataset( ResultFilename_Text.value, 'w' )
    resultCDF.createDimension("time", None)
    v = resultCDF.createVariable("time","u8",("time",))
    v.units = "minutes"
    v = resultCDF.createVariable("lat","f8",("time",))
    v.description = ""
    v.units = "degrees"
    v = resultCDF.createVariable("lon","f8",("time",)) 
    v.description = ""
    v.units = "degrees"
    v = resultCDF.createVariable("MagLat","f8",("time",))
    v.description = "Magnetic Latitude"
    v.units = "degrees"
    v = resultCDF.createVariable("MLT","f8",("time",))
    v.description = "Mean Local Time"
    v.units = "hours"
    v = resultCDF.createVariable("Altitude","f8",("time",))
    v.units = "km"
    v = resultCDF.createVariable("Kp","f8",("time",))
    v.description = "Kp index"
    v.units = "-"    
    v = resultCDF.createVariable("Joule_Heating","f8",("time",))
    v.description = "Joule Heating"
    v.units = "W/m3"
    
    # Read all files in the TIEGCM folder - we will process them all
    DataFilesPath = TIEGCMfilesFolder_Text.value    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*.nc" ) )
    for currentDataFile in AllDataFiles:
        # Read file from disk into memory
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        length_time = CDFroot.variables['Joule Heating'].shape[0]
        length_lev  = CDFroot.variables['Joule Heating'].shape[1]
        length_lat  = CDFroot.variables['Joule Heating'].shape[2]
        length_lon  = CDFroot.variables['Joule Heating'].shape[3]        
        print ("Loading file into memory")
        TIMEs   = CDFroot.variables[         'time'][:, :, :, :]   
        LATs    = CDFroot.variables[          'lat'][:, :, :, :]   
        LONs    = CDFroot.variables[          'lon'][:, :, :, :]   
        MAGLATs = CDFroot.variables[         'MLAT'][:, :, :, :]   
        MLTs    = CDFroot.variables[          'MLT'][:, :, :, :]
        ALTs    = CDFroot.variables[           'ZG'][:, :, :, :] / 100000
        JHs     = CDFroot.variables['Joule Heating'][:, :, :, :]
        KPs     = CDFroot.variables[           'KP'][:]
        
        for idx in range(0, length_time):        
            includeIt = False
            if includeIt:                 
                resultCDF.variables[         "time"][ResultFileIdx] = TIMES[ idx ]
                resultCDF.variables[          "lat"][ResultFileIdx] = LATs[ idx ]
                resultCDF.variables[          "lon"][ResultFileIdx] = LONs[ idx ]
                resultCDF.variables[       "MagLat"][ResultFileIdx] = MAGLATs[ idx ]
                resultCDF.variables[          "MLT"][ResultFileIdx] = MLTs[ idx ]
                resultCDF.variables[     "Altitude"][ResultFileIdx] = ALTs[ idx ]
                resultCDF.variables[           "Kp"][ResultFileIdx] = KPs[ idx ]
                resultCDF.variables["Joule_Heating"][ResultFileIdx] = JHs[ idx ]
    
    # close files, tide up
    resultCDF.close()
    finishSecs = time.time()   
    print( "Finished in", finishSecs-startSecs, "seconds" )
    
    
def createGUI():
    MainTab = w.Tab() 
    #
    CherryPick_Panel = w.VBox()    
    CherryPick_Label = w.HTML(value="<font color='goldenrod'>NOT VALID YET: This application reads several orbit files, chooses data according to the ranges of the region and saves them in a single file.<br>Please select a folder containing orbit files and the result filename and press 'Convert':")
    Convert_CherryPick_Btn = w.Button (description='Convert',tooltip="Convert",)
    Convert_CherryPick_Btn.style.button_color = 'Coral'
    Convert_CherryPick_Btn.on_click( Convert_CherryPick_Btn_Clicked )
    CherryPick_Panel.children = [CherryPick_Label, TIEGCMfilesFolder_Text, Regions_Dropdown, ResultFilename_Text, Convert_CherryPick_Btn]    
    #
    AddFieldsToCSVorbit_Panel = w.VBox()    
    AddFieldsToCSVorbit_Label = w.HTML(value="<font color='goldenrod'>This application adds the following fields to a csv orbit file:<br>Daedalus.Magnetic Latitude, Daedalus.Magnetic Longitude, Daedalus.MLT.<br>Please type the orbit filename and the result filename and press 'Convert':")
    Convert_AddFieldsToCSVorbit_Btn = w.Button (description='Convert',tooltip="Convert",)
    Convert_AddFieldsToCSVorbit_Btn.style.button_color = 'MediumTurquoise'
    Convert_AddFieldsToCSVorbit_Btn.on_click( Convert_AddFieldsToCSVorbit_Btn_Clicked )
    Convert_CoPlotTwoOrbits_Btn = w.Button (description='Plot',tooltip="Plot",)
    Convert_CoPlotTwoOrbits_Btn.style.button_color = 'Gold'
    Convert_CoPlotTwoOrbits_Btn.on_click( CoPlot_TwoOrbits_EventListener )    
    AddFieldsToCSVorbit_Panel.children = [AddFieldsToCSVorbit_Label, OrbitFilename_Text, ResultFilename_Text, Convert_AddFieldsToCSVorbit_Btn, Convert_CoPlotTwoOrbits_Btn]    
    #
    NetCDFcorrection_Panel = w.VBox()
    NetCDFcorrection_Label = w.HTML(value="<font color='goldenrod'>This application corrects all NetCDF files in a directory.<br>Corrections include variable naming, attribute adding etc")
    NetCDFcorrection_Btn   = w.Button (description='Correct all files',tooltip="Correct all files",)
    NetCDFcorrection_Btn.style.button_color = 'Cyan'
    NetCDFcorrection_Btn.on_click( NetCDFcorrection_Btn_EventListener )    
    NetCDFcorrection_Panel.children = [ NetCDFcorrection_Label, NetCDFcorrection_Text, NetCDFcorrection_Btn ]
    #
    MainTab.children = [  AddFieldsToCSVorbit_Panel, NetCDFcorrection_Panel ]
    MainTab.set_title(0, 'Add Mag. Coord.')
    MainTab.set_title(1, 'NetCDF correction')
    #
    return MainTab


def CoPlot_TwoOrbits_EventListener( b ):
    plot_MagneticProperties_ofTwoOrbits( OrbitFilename_Text.value, ResultFilename_Text.value, "Quasi Dipole", "Modified Apex 90" )
    
def plot_MagneticProperties_ofTwoOrbits( orbitfilename1, orbitfilename2, plot_title1, plot_title2):
    print( "Reading files", orbitfilename1, orbitfilename2 )
    # read data from orbit1 file
    fd1 = pd.read_csv( orbitfilename1 ) 
    MAGLATs1 = np.array(fd1["Daedalus.Magnetic Latitude"])
    MAGLONs1 = np.array(fd1["Daedalus.Magnetic Longitude"])
    MAGTIMs1 = np.array(fd1["Daedalus.MLT"])
    # read data from orbit2 file
    fd2 = pd.read_csv( orbitfilename2 ) 
    MAGLATs2 = np.array(fd2["Daedalus.Magnetic Latitude"])
    MAGLONs2 = np.array(fd2["Daedalus.Magnetic Longitude"])
    MAGTIMs2 = np.array(fd2["Daedalus.MLT"])
    
    # make data smaller so that they can be plotted
    num_of_points_to_plot = 8000
    plot_step = int( len(MAGLATs1) / num_of_points_to_plot )
    MAGLATs1 = MAGLATs1[0::plot_step]
    MAGLONs1 = MAGLONs1[0::plot_step]
    MAGTIMs1 = MAGTIMs1[0::plot_step]
    MAGLATs2 = MAGLATs2[0::plot_step]
    MAGLONs2 = MAGLONs2[0::plot_step]
    MAGTIMs2 = MAGTIMs2[0::plot_step]
    print( "plotting totally", num_of_points_to_plot, "points. That is one every", plot_step )
    '''
    MAGLATs1 = MAGLATs1[0:num_of_points_to_plot]
    MAGLONs1 = MAGLONs1[0:num_of_points_to_plot]
    MAGTIMs1 = MAGTIMs1[0:num_of_points_to_plot]
    MAGLATs2 = MAGLATs2[0:num_of_points_to_plot]
    MAGLONs2 = MAGLONs2[0:num_of_points_to_plot]
    MAGTIMs2 = MAGTIMs2[0:num_of_points_to_plot]
    '''
    
    
    # plot the Magnetic Latitudes of the two orbits together
    fig = go.Figure()
    fig.add_trace( go.Scatter(name=plot_title1, x=list(range(0,len(MAGLATs1))), y=MAGLATs1, mode='markers', marker_size=3, marker_color="red"  ) )
    fig.add_trace( go.Scatter(name=plot_title2, x=list(range(0,len(MAGLATs2))), y=MAGLATs2, mode='markers', marker_size=3, marker_color="blue"  ) )
    fig.update_layout( title="Magnetic Latitudes comparison", width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
    fig.update_xaxes( title="time step number" )
    fig.update_yaxes( title="Magnetic Latitude") 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
    return
    # plot the Magnetic Longitudes of the two orbits together
    fig = go.Figure()
    fig.add_trace( go.Scatter(name=plot_title1, x=list(range(0,len(MAGLONs1))), y=MAGLONs1, mode='markers', marker_size=3, marker_color="red"  ) )
    fig.add_trace( go.Scatter(name=plot_title2, x=list(range(0,len(MAGLONs2))), y=MAGLONs2, mode='markers', marker_size=3, marker_color="blue"  ) )
    fig.update_layout( title="Magnetic Longitudes comparison", width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
    fig.update_xaxes( title="time step number" )
    fig.update_yaxes( title="Magnetic Longitude") 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
    # plot the Magnetic Local Times of the two orbits together
    fig = go.Figure()
    fig.add_trace( go.Scatter(name=plot_title1, x=list(range(0,len(MAGTIMs1))), y=MAGTIMs1, mode='markers', marker_size=3, marker_color="red"  ) )
    fig.add_trace( go.Scatter(name=plot_title2, x=list(range(0,len(MAGTIMs2))), y=MAGTIMs2, mode='markers', marker_size=3, marker_color="blue"  ) )
    fig.update_layout( title="Magnetic Local Time comparison", width=1000, height=900, legend_orientation="h", legend= {'itemsizing': 'constant'}) 
    fig.update_xaxes( title="time step number" )
    fig.update_yaxes( title="Magnetic Local Time") 
    plotly.offline.init_notebook_mode(connected=True)
    plotly.offline.iplot(fig)
        
        
OrbitFilename_Text.value = "/home/NAS/Data_Files/OrbitData/x.csv"
ResultFilename_Text.value = "/home/NAS/Data_Files/OrbitData/z.csv"
OrbitFilename_Text.value = "/home/NAS/Data_Files/OrbitData/DAED_ORB_Lifetime_INC80_RAAN00_CAMP02_BL000_Srt010Hz_115km_c.csv"
ResultFilename_Text.value = "/home/NAS/Data_Files/OrbitData/DAED_ORB_Lifetime_INC80_RAAN00_CAMP02_BL000_Srt010Hz_115km_c.QD.csv"
NetCDFcorrection_Text.value = "/home/NAS/TIEGCM_DATA/TIEGCM_Lifetime_2015_to_2018_JH_QD/"
createGUI() 
#print(  calendar.timegm ( datetime.strptime("25 Oct 2015 03:59:50.000"[0:24], '%d %b %Y %H:%M:%S.%f').utctimetuple() ) )
#print(  calendar.timegm ( datetime.strptime("25 Oct 2015 04:00:00.000"[0:24], '%d %b %Y %H:%M:%S.%f').utctimetuple() ) ) 
#print(  datetime.timestamp ( datetime.strptime("25 Oct 2015 04:00:01.000"[0:24], '%d %b %Y %H:%M:%S.%f') ) )
#print(  datetime.timestamp ( datetime(2008, 1, 1, 0, 0, 0, 0) ) )
#print(  calendar.timegm(datetime(2008, 1, 1, 0, 0, 0, 0).utctimetuple())  )



Reading /home/NAS/Data_Files/OrbitData/DAED_ORB_Lifetime_INC80_RAAN00_CAMP03_BL000_001sec_115km.csv 
Writing /home/NAS/Data_Files/OrbitData/DAED_ORB_Lifetime_INC80_RAAN00_CAMP03_BL000_001sec_115km.QD.csv
